# Quickstart

A quickstart for FIOS.

The goal of this document is to lay out an API for a developing package

In [ ]:
import fios

## IO
Read in a single file

In [ ]:
st = fios.read_fiber('path/to/fiber.hdf5')
# access the first trace (contiguous data array of the same ID)
tr = st[0]

Index and query data from a directory of fiber files (called a Spool)

In [ ]:
spool = fios.Spool('path/to/directory').update_index()

get a stream from interogator AA for a specifc time range

In [ ]:
tr = spool.get_fiber(id='AA', time=('2021-01-15T10:01:12', '2021-01-15T10:03:12'))[0]

Put data into a spool

In [ ]:
spool.put_fiber(tr)

## Selecting Data in Traces

Trim trace to only include 5 seconds from start to 5 seconds from end, exclude first 100m stretch

In [ ]:
tr1 = tr.select(time=(5, -5), distance=(100, None))

see trace attributes|

In [ ]:
print(tr1.attrs)  # {'time_min': datetime64('2021-021-15T10:01:17'), id: 'AA', ...}

See dimension names of trace (eg, usually 'time', 'distance') 

In [ ]:
print(tr1.dims)

Modify attributes, return new trace

In [ ]:
# set new starttime, will update endtime, etc. 
tr2 = tr1.update_attrs(starttime='2022-01-01')

Create new trace with identical attributes but different data

In [ ]:
data = tr2.data + 10  # add 10 to each data element
tr3 = tr2.new(data=data)

Create new trace with different distance attributes

In [ ]:
distance = tr2.coords.distance / 10
tr4 = tr3.new(coords={'distance': distance, 'time': tr2.coords.time})

## Trace Data Processing

Examaple of method chaining to form a data processing pipeline

In [ ]:
tr5 = (
    tr4.decimate(10, low_pass=True)  # decimate 10x, lowpass first. 
    .detrend(dim='time', linear=True)  # detrend on time axis
    .pass_filter(frequency=(None, 100))  # apply 100 Hz Highpass
    .stop_filter(frequency=(50, 70))  # followed by 50 to 70 notch filter
    .compute()  # Needed for delayed resolution on distributed processing?, or is context manager better?
)

## Trace Transformation
Examples of transforming trace data

In [ ]:
tr_fft = tr5.transform.rfft(dim='time')  # fft along time dimension
print(tr_fft.dims)  # dims are now (frequency, distance)

In [ ]:
tr_ifft = tr5.transform.irfft(dim='time')  # transform back to time domain

Also, 2D transforms (eg fk)

In [ ]:
tr_fk = tr5.transform.fk()

And more complicated transforms...

In [ ]:
tr5.transform.marr_wavelette()

## Writing trace functions/methods
Example of writing processing methods for trace. The require decorator allows the declaration and enforcement of expectations of the input trace. 

In [ ]:
# a decorator to declare what the function requires of the trace
@fios.require(dims=('time', 'distance'))  # raise if time/distance are not in dimensions
def some_function(tr: fios.Trace, arg1=10) -> fios.Trace:
    """An example of a function to do something to a trace."""
    data = tr.data
    time = tr.coords.time
    new_data = data / np.std(data, axis=1)
    return tr.new(data=new_data)
    

## Visualization
Make a waterfall plot, return figure. `

In [ ]:
fig = tr5.viz.waterfall_plot()

Make a shot gather plot

In [ ]:
fig = tr5.viz.shot_gather()

## Streams
Streams are a collection of traces. Most trace methods are also found on streams but are performed in parallel for each trace.

In [ ]:
st = spool.read_fiber()  # deplayed, out of core, read of all files in directory

In [ ]:
st1 = (
    st.merge()  # merge all contiguous blocks with the same id
    .rechunk(time=10, distance=300)  # rechunk traces in memory
    .decimate(5)  # decimate all streams
    .filter(frequency=(1, 200), wavelength=(None, 100))  # wavelength and frequency filter
)

No computations have been performed on the stream yet; it is all delayed computations.
Specify batches and batch over stream data

In [ ]:
for sub_st in st1.batch(chunks=10):
    st_new = (
        sub_st.update_attrs(id='AA_processed')
        .some_other_method()
        .compute()  # resolves delayed computation graph now
    )
    spool.put_fiber(st_new)  # put processed fiber data back into bank with new ID
